In [ ]:
import os
import numpy as np
import open3d as o3d
import pandas as pd
from scipy.spatial import ConvexHull
from scipy.spatial.distance import pdist
from skimage.measure import EllipseModel
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.interpolate import interp1d
from scipy.interpolate import interp1d
from scipy.integrate import simps
df_new = pd.DataFrame(
)

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)
    return inlier_cloud
    # print("Showing outliers (red) and inliers (gray): ")
    # outlier_cloud.paint_uniform_color([1, 0, 0])
    # inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    # o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

def estimate_curvature(points, k=30):
        curvatures = []
        nbrs = NearestNeighbors(n_neighbors=k).fit(points)
        _, indices = nbrs.kneighbors(points)

        for i, idx in enumerate(indices):
            neighbors = points[idx]
            cov = np.cov(neighbors.T)
            eigvals, _ = np.linalg.eigh(cov)
            curvature = eigvals[0] / np.sum(eigvals)
            curvatures.append(curvature)
        return np.mean(curvatures)
# Lista de arquivos .ply
arquivos_ovos = os.listdir(r'C:\Users\mhmon\ovos-pv\ovos3')

# Carregar dados reais
df = pd.read_csv(r'C:\Users\mhmon\ovos-pv\Teste_Peso.csv')
# df = df.dropna(subset=["B", "L "])

altura_real_lista = []
largura_real_lista = []
altura_pred_lista = []
largura_pred_lista = []
alturamax_pred_lista = []
features = {}
# Loop para testar cada ovo
for i, row in df.iterrows():
    escala = 1.07
    escala_b = 1.04

    nome = row['nome']
    # nome = 'ovo'+str(num)
    arquivo = f'C:\\Users\\mhmon\\ovos-pv\\ovos3\\{nome}'
    numero_ovo = nome.replace('ovo', '')  # Extrai o número do ovo
    numero_ovo = int(numero_ovo.replace('.ply', ''))  # Extrai o número do ovo
    print(numero_ovo)
    if (not(numero_ovo <= 1000)):  # Filtra os ovos dentro do intervalo desejado
        continue

    # Escalas diferentes para bandejas diferentes
    if(numero_ovo>200 and numero_ovo < 400):
            escala = 1.06
    if(numero_ovo>600 and numero_ovo < 690):
        escala = 1.05
    if(numero_ovo>690 and numero_ovo < 801):
        escala = 1.1
    if(numero_ovo>720 and numero_ovo < 790):
        escala = 1.12
        escala_b = 1.06
    if not os.path.exists(arquivo):  # Verifica se o arquivo existe
        print(f"Arquivo {arquivo} não encontrado, pulando...")
        continue
    print(arquivo)
    # Carregar nuvem de pontos
    pcd = o3d.io.read_point_cloud(arquivo)
    points = np.asarray(pcd.points)


    # Aplicar PCA para encontrar os eixos principais
    mean = np.mean(points, axis=0)
    points_centered = points - mean
    U, S, Vt = np.linalg.svd(points_centered)


    rotated_points = np.dot(points_centered, Vt.T)

    # # Criar nova nuvem de pontos alinhada
    pcd_aligned = o3d.geometry.PointCloud()
    pcd_aligned.points = o3d.utility.Vector3dVector(rotated_points)
    # Altura (Y) e Largura (X)
    altura_y = row["L_estimado"]
    largura_x = row["B_estimado"]

    # bounds = np.ptp(points, axis=0)  # range total por dimensão
    # volume_bbox = bounds[0] * bounds[1] * bounds[2]
    # densidade = len(points) / volume_bbox

    # print(volume_bbox)
    # print(densidade)




    # Eixo Y = altura, eixo X = raio lateral (largura em X ou Z)
    r = []
    y = []

    for yi in np.linspace(np.min(points[:, 1]), np.max(points[:, 1]), 100):
        faixa = points[(points[:, 1] > yi - 0.5) & (points[:, 1] < yi + 0.5)]
        if len(faixa) == 0:
            continue
        raio = (np.max(faixa[:, 0]) - np.min(faixa[:, 0])) / 2
        r.append(raio)
        y.append(yi)

    # Interpolação
    f = interp1d(y, r, kind='linear', fill_value="extrapolate")
    fprime = np.gradient(r, y)

    '''
    CALCULO REVOLUÇAO 1
    '''
    integrando = [f(y[i]) * np.sqrt(1 + fprime[i] ** 2) for i in range(len(y))]
    area_revolucao_1 = 2 * np.pi * simps(integrando, y)
    # Integral para superfície de revolução: 2π ∫ r(y) * √(1 + r'(y)²) dy
    # Raio com escalab

    '''
    CALCULO REVOLUÇAO 2
    '''
    r_scaled = [raio * escalab for raio in r]

    # Altura com escala
    y_scaled = [yi * escala for yi in y]

    fprime_scaled = np.gradient(r_scaled, y_scaled)

    integrando = [r_scaled[i] * np.sqrt(1 + fprime_scaled[i] ** 2) for i in range(len(y))]
    area_revolucao_2 = 2 * np.pi * simps(integrando, y_scaled)  # unidade: mm² se escala for em mm

    '''
    CALCULO CURVATURA
    '''
    curvatura_media = estimate_curvature(points)
    print(curvatura_media)


    altura_total = np.max(points[:, 1]) - np.min(points[:, 1])
    altura_3quartos = np.min(points[:, 1]) + 0.25 * altura_total

    tolerancia = 0.2  # menor faixa vertical para definir plano de corte
    slice_quarto = points[(points[:, 1] > altura_3quartos - tolerancia) & (points[:, 1] < altura_3quartos + tolerancia)]

    diam_x = np.max(slice_quarto[:, 0]) - np.min(slice_quarto[:, 0])
    diam_z = np.max(slice_quarto[:, 2]) - np.min(slice_quarto[:, 2])
    diam_quarto = min(diam_x, diam_z) *escalab # melhor estimativa do diâmetro




    nova_linha = pd.DataFrame([{
        'nome': nome,
        "sup_rev": area_revolucao_1,
        "diam_4": diam_quarto,
        "curvatura_media":curvatura_media

    }])


    df_new = pd.concat([df_new, nova_linha], ignore_index=True)



In [ ]:
df

,nome,L_estimado,L,B,B_estimado,linha,S_OVO,P_OVO,D/4,VOL_OVO,bounding_box_area,pca_ratio_2_0,area_proj_2d,shape_compactness,CL_PESO,CLA_I_FOR,sup_rev,sup_rev_escala,diam_4,curvatura_media
0,ovo454.ply,59.71,60.70,46.71,46.40,3,68.10,70.5,41.71,69.08,27.70,10.92,15.65,56.48,Jumbo,Redondo,75.18,83.66,28.08,28.68
1,ovo450.ply,59.24,59.15,45.21,46.30,2,64.73,58.1,37.87,63.39,27.43,11.01,15.44,56.30,Extra,Redondo,75.27,83.76,24.96,25.93
2,ovo870.ply,62.62,60.53,45.55,45.65,1,66.24,67.4,40.49,65.77,28.59,9.08,15.80,55.26,Extra,Padrão,78.27,87.10,21.84,27.08
3,ovo39.ply,60.51,58.99,44.60,45.21,3,63.72,65.6,37.87,61.65,27.36,9.70,15.27,55.81,Extra,Padrão,73.06,81.30,26.00,29.47
4,ovo660.ply,57.56,57.47,44.98,44.76,3,63.15,63.4,38.35,61.01,25.77,7.87,15.41,59.83,Extra,Redondo,73.69,80.47,21.84,29.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,ovo80.ply,59.58,58.30,42.91,43.47,2,60.76,59.4,37.79,56.73,25.90,9.42,15.11,58.34,Extra,Padrão,71.64,79.72,20.80,24.85
679,ovo909.ply,59.67,58.80,43.82,43.57,1,62.44,60.1,39.26,59.48,26.00,7.51,15.20,58.48,Extra,Padrão,71.99,80.11,18.72,26.58
680,ovo71.ply,57.61,58.90,44.27,45.08,2,63.17,63.7,38.92,60.72,25.97,9.33,15.52,59.75,Extra,Padrão,73.78,82.10,19.76,25.58
681,ovo604.ply,56.87,57.12,45.92,46.47,1,64.25,64.7,39.45,63.00,26.43,11.51,15.50,58.66,Extra,Redondo,76.03,83.03,19.76,27.26


In [ ]:
df_new.to_csv('Teste_V4.csv', index=False)